# Indexes and Ranges

* Getting more expressiveness around working with indexed data structures
* The new operators (^ and ..) are syntactic sugar. The functionality can be implemented by explicit calls to System.Index and System.Range factory methods
* Implicit Index support. Pattern based approach to get indexes and ranges out of the box for certain scenarios
* No special IL representation

```csharp
namespace System
{
    public readonly struct Index
    {
        public Index(int value, bool fromEnd);
        //...
    }
}
namespace System
{
    public readonly struct Range
    {
        public Range(System.Index start, System.Index end);
        public static Range StartAt(System.Index start);
        public static Range EndAt(System.Index end);
        public static Range All { get; }
    }
}

System.Index operator ^(int fromEnd);
System.Range operator ..(Index start = 0, Index end = ^0);
```


In [10]:
string[] names =
{
    "Archimedes", "Pythagoras", "Euclid", "Socrates", "Plato"
};

display("0..^0 == .. == ..^0 == 0..");
display(names[..]);

display("^2");
display(names[^2]);

display("^2..^0");
display(names[^2..^0]);

0..^0 == .. == ..^0 == 0..

index,value
0,Archimedes
1,Pythagoras
2,Euclid
3,Socrates
4,Plato


^2

Socrates

^2..^0

index,value
0,Socrates
1,Plato


In [18]:
var array = new int[] { 1, 2, 3, 4, 5 };

display(array);

display("[2..^3] == array[new Range(2, new Index(3, fromEnd: true))]");
var slice1 = array[2..^3];    // array[new Range(2, new Index(3, fromEnd: true))]
display(slice1);

display("[..^3] == array[Range.EndAt(new Index(3, fromEnd: true))]");
var slice2 = array[..^3];     // array[Range.EndAt(new Index(3, fromEnd: true))]
display(slice2);

display("[2..] == array[Range.StartAt(2)]");
var slice3 = array[2..];      // array[Range.StartAt(2)]
display(slice3);

display("[..] == array[Range.All]");
var slice4 = array[..];       // array[Range.All]
display(slice4);

display(array[^10]);

index,value
0,1
1,2
2,3
3,4
4,5


[2..^3] == array[new Range(2, new Index(3, fromEnd: true))]

(empty)

[..^3] == array[Range.EndAt(new Index(3, fromEnd: true))]

index,value
0,1
1,2


[2..] == array[Range.StartAt(2)]

index,value
0,3
1,4
2,5


[..] == array[Range.All]

index,value
0,1
1,2
2,3
3,4
4,5


Unhandled exception: System.IndexOutOfRangeException: Index was outside the bounds of the array.
   at Submission#21.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

## Demo

In [29]:
public class StatInfo
{
    public string Label {get; set;}
    public int Min {get; set;}
    public int Max {get; set;}
    public double Average {get; set;}
}

int[] sequence = Sequence(1000);

IEnumerable<StatInfo> LeftToRightAverage()
{
    for(int start = 0; start < sequence.Length; start += 100)
    {
        Range r = start..(start+10);
        var (min, max, average) = MovingAverage(sequence, r);
        // Console.WriteLine($"From {r.Start} to {r.End}:    \tMin: {min},\tMax: {max},\tAverage: {average}");
        yield return new StatInfo() 
        {
            Label = $"From {r.Start} to {r.End}",
            Min = min, 
            Max = max,
            Average = average,
        };
    }
}

display(LeftToRightAverage().ToList());

IEnumerable<StatInfo> RightToLeftAverage()
{
    for (int start = 0; start < sequence.Length; start += 100)
    {
        Range r = ^(start + 10)..^start;
        var (min, max, average) = MovingAverage(sequence, r);
        // Console.WriteLine($"From {r.Start} to {r.End}:  \tMin: {min},\tMax: {max},\tAverage: {average}");
        yield return new StatInfo() 
        {
            Label = $"From {r.Start} to {r.End}",
            Min = min, 
            Max = max,
            Average = average,
        };
    }
}

display(RightToLeftAverage().ToList());

(int min, int max, double average) MovingAverage(int[] subSequence, Range range) =>
    (
        subSequence[range].Min(),
        subSequence[range].Max(),
        subSequence[range].Average()
    );

int[] Sequence(int count) => Enumerable.Range(0, count).Select(x => (int)(Math.Sqrt(x) * 100)).ToArray();

index,Label,Min,Max,Average
0,From 0 to 10,0,300,192.7
1,From 100 to 110,1000,1044,1021.6
2,From 200 to 210,1414,1445,1429.5
3,From 300 to 310,1732,1757,1744.4
4,From 400 to 410,2000,2022,2010.6
5,From 500 to 510,2236,2256,2245.6
6,From 600 to 610,2449,2467,2458
7,From 700 to 710,2645,2662,2653.7
8,From 800 to 810,2828,2844,2835.9
9,From 900 to 910,3000,3014,3006.9


index,Label,Min,Max,Average
0,From ^10 to ^0,3146,3160,3153.1
1,From ^110 to ^100,2983,2998,2990.2
2,From ^210 to ^200,2810,2826,2818.2
3,From ^310 to ^300,2626,2643,2634.8
4,From ^410 to ^400,2428,2447,2437.9
5,From ^510 to ^500,2213,2233,2223.2
6,From ^610 to ^600,1974,1997,1985.5
7,From ^710 to ^700,1702,1729,1715.5
8,From ^810 to ^800,1378,1410,1394.2
9,From ^910 to ^900,948,994,971.3


# Reference

* <https://docs.microsoft.com/en-us/dotnet/csharp/tutorials/ranges-indexes>
* <https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/proposals/csharp-8.0/ranges>
